In [2]:
import os 
import pandas as pd
import numpy as np

import IPython.display as ipd
import warnings
import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils

import processing
import event_manager
import model_training
import epoching

warnings.filterwarnings('ignore')

In [3]:
audio_folder = '../../data/ICBHI_final_database'
list_audio_files = processing.get_list_recording(audio_folder)
print('Found {} recording files'.format(len(list_audio_files)))

Found 920 recording files


In [4]:
label_list = []
epochs_list = []
n_fft=400
win_length = 0.100
list_series = list()
for audio_file in tqdm(list_audio_files):
    data_file = processing.load_file_from_recording_name(audio_folder,audio_file)
    data_filtered,df_label = processing.preprocess_data(data_file,annotations='text')
    data_feature= processing.apply_stft(data_filtered,n_fft=400,center=False)
    epochs_,label_ = epoching.get_epoching_from_label(data_feature,df_label,mean=False)
    epochs_list.append(epochs_)
    label_list.append(label_)

100%|██████████| 920/920 [03:50<00:00,  3.98it/s]


In [5]:
epoch_final, df_label = model_training.prepare_data_time_for_modeling(epochs_list,label_list,list_audio_files)

In [8]:
epoch_final.shape

(6898, 201, 646)

In [10]:
import pickle 
X = {'feature': epoch_final,
     'label':df_label}
file = '../../data/data_model/data_sound.pickle'
with open(file, 'wb') as handle:
    pickle.dump(X, handle)